In [35]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

df = pd.read_csv('2024_Wimbledon_featured_matches_washed.csv')

In [36]:
# 检查关键列及其含义
print("关键列检查:")
print("\nserver列 - 发球手: ", df['server'].unique())
print("serve_no列 - 第几发: ", sorted(df['serve_no'].dropna().unique()))
print("\nbreak_pt相关列:")
print("  p1_break_pt - player1的破发点机会: ", df['p1_break_pt'].sum(), "次")
print("  p1_break_pt_won - player1破发成功: ", df['p1_break_pt_won'].sum(), "次")
print("\npoint_victor - 得分者: ", df['point_victor'].unique())
print("\n前5行的关键信息:")
print(df[['match_id', 'server', 'serve_no', 'point_victor', 'p1_ace', 'p2_ace', 'p1_winner', 'p2_winner', 
          'p1_double_fault', 'p2_double_fault', 'p1_break_pt', 'p1_break_pt_won']].head())

关键列检查:

server列 - 发球手:  [1 2]
serve_no列 - 第几发:  [np.int64(1), np.int64(2)]

break_pt相关列:
  p1_break_pt - player1的破发点机会:  277 次
  p1_break_pt_won - player1破发成功:  101 次

point_victor - 得分者:  [2 1]

前5行的关键信息:
              match_id  server  serve_no  point_victor  p1_ace  p2_ace  \
0  2023-wimbledon-1301       1         2             2       0       0   
1  2023-wimbledon-1301       1         1             1       0       0   
2  2023-wimbledon-1301       1         1             2       0       0   
3  2023-wimbledon-1301       1         1             1       0       0   
4  2023-wimbledon-1301       1         1             1       1       0   

   p1_winner  p2_winner  p1_double_fault  p2_double_fault  p1_break_pt  \
0          0          0                0                0            0   
1          0          0                0                0            0   
2          0          0                0                0            0   
3          1          0                0             

In [37]:
# 确定M值：找到不会产生大量缺失值的最小M
print("="*60)
print("确定滑动窗口大小M")
print("="*60)

# 按match_id排序，保证顺序
df_sorted = df.sort_values(['match_id', 'point_no']).reset_index(drop=True)

# 测试不同的M值，看看缺失比例
test_M_values = [3, 5, 10, 15, 20, 25]

print("\n测试不同M值下的可用数据比例:")
print("M值\t第1发进球数\t缺失比例")
print("-" * 60)

for M in test_M_values:
    # 计算滑动窗口内第1发是否进球（serve_no==1且point_victor存在）
    first_serve_in = (df_sorted['serve_no'] == 1).astype(int)
    
    # 对每一行，计算最近M个球中第1发进球情况
    moving_avg = first_serve_in.rolling(window=M, min_periods=1).sum()
    
    # 理想情况：计算缺失值
    # serve_no为NaN的行表示数据不完整
    serve_no_valid = df_sorted['serve_no'].notna()
    valid_count = serve_no_valid.sum()
    missing_pct = (1 - valid_count / len(df_sorted)) * 100
    
    print(f"{M}\t{moving_avg.sum():.0f}\t\t{missing_pct:.2f}%")

print("\n推荐M=10（平衡数据量和缺失值）")

确定滑动窗口大小M

测试不同M值下的可用数据比例:
M值	第1发进球数	缺失比例
------------------------------------------------------------
3	13968		0.00%
5	23275		0.00%
10	46528		0.00%
15	69762		0.00%
20	92984		0.00%
25	116184		0.00%

推荐M=10（平衡数据量和缺失值）


In [38]:
# 计算基于最近M个球的指标
print("\n" + "="*60)
print("计算最近M个球的指标 (M=10)")
print("="*60)

M = 10

# 按match_id和point_no排序
df_sorted = df.sort_values(['match_id', 'point_no']).reset_index(drop=True)

# 初始化新的指标列
new_columns = {
    'p1_1st_serve_in_rate': 0.0,
    'p2_1st_serve_in_rate': 0.0,
    'p1_1st_serve_won_rate': 0.0,
    'p2_1st_serve_won_rate': 0.0,
    'p1_return_won_rate': 0.0,
    'p2_return_won_rate': 0.0,
    'p1_break_count': 0,
    'p2_break_count': 0,
    'p1_double_fault_count': 0,
    'p2_double_fault_count': 0,
    'p1_aces_count': 0,
    'p2_aces_count': 0,
    'p1_winner_count': 0,
    'p2_winner_count': 0,
    'p1_unforced_error_count': 0,
    'p2_unforced_error_count': 0,
}

for col in new_columns:
    df_sorted[col] = 0.0 if 'rate' in col else 0

# 对每一行计算基于最近M个球（包括当前球）的指标
print("\n计算指标中...")

for idx in range(len(df_sorted)):
    # 确定窗口范围：最多往前M个球，但不跨越match_id
    match_id = df_sorted.loc[idx, 'match_id']
    
    # 找到同一比赛内最近M个球的范围
    window_start = max(0, idx - M + 1)
    # 确保不跨越match_id
    while window_start > 0 and df_sorted.loc[window_start - 1, 'match_id'] == match_id:
        window_start -= 1
    
    window_df = df_sorted.iloc[window_start:idx + 1]
    
    # Player 1 指标（player1发球的情况）
    p1_serve_mask = window_df['server'] == 1
    p1_serve_rows = window_df[p1_serve_mask]
    
    # 1st serve in rate (第1发进球比例)
    p1_1st_serve = p1_serve_rows[p1_serve_rows['serve_no'] == 1]
    if len(p1_1st_serve) > 0:
        # 假设所有serve_no==1且serve_width不为全0表示进球
        p1_1st_in = len(p1_1st_serve[p1_1st_serve['serve_width_B'].fillna(False) | 
                                       p1_1st_serve['serve_width_BC'].fillna(False) |
                                       p1_1st_serve['serve_width_BW'].fillna(False) |
                                       p1_1st_serve['serve_width_C'].fillna(False) |
                                       p1_1st_serve['serve_width_W'].fillna(False)])
        df_sorted.loc[idx, 'p1_1st_serve_in_rate'] = round((p1_1st_in / len(p1_1st_serve)) * 100, 2)
    
    # 1st serve won rate (第1发得分比例)
    if len(p1_1st_serve) > 0:
        p1_1st_won = len(p1_1st_serve[p1_1st_serve['point_victor'] == 1])
        df_sorted.loc[idx, 'p1_1st_serve_won_rate'] = round((p1_1st_won / len(p1_1st_serve)) * 100, 2)
    
    # Return won rate (接发球得分比例)
    p1_return_rows = window_df[window_df['server'] == 2]  # p1接发球时server==2
    if len(p1_return_rows) > 0:
        p1_return_won = len(p1_return_rows[p1_return_rows['point_victor'] == 1])
        df_sorted.loc[idx, 'p1_return_won_rate'] = round((p1_return_won / len(p1_return_rows)) * 100, 2)
    
    # Break count (破发成功次数 - 最近M行中的总数)
    df_sorted.loc[idx, 'p1_break_count'] = window_df['p1_break_pt_won'].sum()
    
    # Count统计（最近M个球中的总数 - 直接在window_df中统计）
    df_sorted.loc[idx, 'p1_double_fault_count'] = window_df['p1_double_fault'].sum()
    df_sorted.loc[idx, 'p1_aces_count'] = window_df['p1_ace'].sum()
    df_sorted.loc[idx, 'p1_winner_count'] = window_df['p1_winner'].sum()
    df_sorted.loc[idx, 'p1_unforced_error_count'] = window_df['p1_unf_err'].sum()
    
    # Player 2 指标（同理）
    p2_serve_mask = window_df['server'] == 2
    p2_serve_rows = window_df[p2_serve_mask]
    
    # 1st serve in rate
    p2_1st_serve = p2_serve_rows[p2_serve_rows['serve_no'] == 1]
    if len(p2_1st_serve) > 0:
        p2_1st_in = len(p2_1st_serve[p2_1st_serve['serve_width_B'].fillna(False) | 
                                       p2_1st_serve['serve_width_BC'].fillna(False) |
                                       p2_1st_serve['serve_width_BW'].fillna(False) |
                                       p2_1st_serve['serve_width_C'].fillna(False) |
                                       p2_1st_serve['serve_width_W'].fillna(False)])
        df_sorted.loc[idx, 'p2_1st_serve_in_rate'] = round((p2_1st_in / len(p2_1st_serve)) * 100, 2)
    
    # 1st serve won rate
    if len(p2_1st_serve) > 0:
        p2_1st_won = len(p2_1st_serve[p2_1st_serve['point_victor'] == 2])
        df_sorted.loc[idx, 'p2_1st_serve_won_rate'] = round((p2_1st_won / len(p2_1st_serve)) * 100, 2)
    
    # Return won rate
    p2_return_rows = window_df[window_df['server'] == 1]  # p2接发球时server==1
    if len(p2_return_rows) > 0:
        p2_return_won = len(p2_return_rows[p2_return_rows['point_victor'] == 2])
        df_sorted.loc[idx, 'p2_return_won_rate'] = round((p2_return_won / len(p2_return_rows)) * 100, 2)
    
    # Break count (破发成功次数 - 最近M行中的总数)
    df_sorted.loc[idx, 'p2_break_count'] = window_df['p2_break_pt_won'].sum()
    
    # Count统计（最近M个球中的总数 - 直接在window_df中统计）
    df_sorted.loc[idx, 'p2_double_fault_count'] = window_df['p2_double_fault'].sum()
    df_sorted.loc[idx, 'p2_aces_count'] = window_df['p2_ace'].sum()
    df_sorted.loc[idx, 'p2_winner_count'] = window_df['p2_winner'].sum()
    df_sorted.loc[idx, 'p2_unforced_error_count'] = window_df['p2_unf_err'].sum()
    
    if (idx + 1) % 1000 == 0:
        print(f"  已处理 {idx + 1}/{len(df_sorted)} 行")

print(f"\n✓ 指标计算完成！添加了 {len(new_columns)} 个新指标")
print(f"\n新增指标列:")
for col in sorted(new_columns.keys()):
    print(f"  - {col}")


计算最近M个球的指标 (M=10)

计算指标中...
  已处理 1000/7284 行
  已处理 2000/7284 行
  已处理 3000/7284 行
  已处理 4000/7284 行
  已处理 5000/7284 行
  已处理 6000/7284 行
  已处理 7000/7284 行

✓ 指标计算完成！添加了 16 个新指标

新增指标列:
  - p1_1st_serve_in_rate
  - p1_1st_serve_won_rate
  - p1_aces_count
  - p1_break_count
  - p1_double_fault_count
  - p1_return_won_rate
  - p1_unforced_error_count
  - p1_winner_count
  - p2_1st_serve_in_rate
  - p2_1st_serve_won_rate
  - p2_aces_count
  - p2_break_count
  - p2_double_fault_count
  - p2_return_won_rate
  - p2_unforced_error_count
  - p2_winner_count


In [39]:
# 删除已转化的原始列
print("\n" + "="*60)
print("清理数据：删除已转化的原始列")
print("="*60)

# 需要删除的列（已经转化成指标的列）
cols_to_drop = [
    # 已转化为count的列
    'p1_ace', 'p2_ace',
    'p1_winner', 'p2_winner',
    'p1_double_fault', 'p2_double_fault',
    'p1_unf_err', 'p2_unf_err',
    
    # 已转化为rate的列
    'p1_break_pt', 'p2_break_pt',
    'p1_break_pt_won', 'p2_break_pt_won',
    'p1_break_pt_missed', 'p2_break_pt_missed',
    
    # 其他可以删除的列（网位得分相关，已转化为指标）
    'p1_net_pt', 'p2_net_pt',
    'p1_net_pt_won', 'p2_net_pt_won',
    
    # 保留的列（需要继续用于其他分析）
    # - match_id, player1, player2: ID信息
    # - set_no, game_no, point_no: 比赛进度
    # - server, serve_no: 发球信息
    # - point_victor, game_victor, set_victor: 得分信息
    # - winner_shot_type_*: one-hot编码的shot类型
    # - serve_width_*, serve_depth_*, return_depth_*: one-hot编码的位置
    # - p1_points_won, p2_points_won: 积分统计
    # - rally_count, speed_mph: 比赛动态信息
    # - distance_run: 跑动距离
]

# 检查这些列是否存在
existing_drop_cols = [col for col in cols_to_drop if col in df_sorted.columns]

print(f"\n将删除 {len(existing_drop_cols)} 列:")
for col in existing_drop_cols:
    print(f"  - {col}")

df_final = df_sorted.drop(columns=existing_drop_cols)

print(f"\n删除前列数: {len(df_sorted.columns)}")
print(f"删除后列数: {len(df_final.columns)}")
print(f"减少列数: {len(df_sorted.columns) - len(df_final.columns)}")

print(f"\n最终数据集形状: {df_final.shape}")
print(f"\n最终列名:")
for i, col in enumerate(df_final.columns, 1):
    col_type = '(指标)' if any(x in col for x in ['rate', 'count']) else ''
    print(f"{i:2d}. {col:30s} {col_type}")

# 验证缺失值
print(f"\n缺失值统计:")
missing_summary = df_final.isnull().sum()
if missing_summary.sum() > 0:
    print(missing_summary[missing_summary > 0])
else:
    print("✓ 无缺失值")


清理数据：删除已转化的原始列

将删除 18 列:
  - p1_ace
  - p2_ace
  - p1_winner
  - p2_winner
  - p1_double_fault
  - p2_double_fault
  - p1_unf_err
  - p2_unf_err
  - p1_break_pt
  - p2_break_pt
  - p1_break_pt_won
  - p2_break_pt_won
  - p1_break_pt_missed
  - p2_break_pt_missed
  - p1_net_pt
  - p2_net_pt
  - p1_net_pt_won
  - p2_net_pt_won

删除前列数: 63
删除后列数: 45
减少列数: 18

最终数据集形状: (7284, 45)

最终列名:
 1. match_id                       
 2. player1                        
 3. player2                        
 4. set_no                         
 5. game_no                        
 6. point_no                       
 7. winner_shot_type_0             
 8. winner_shot_type_B             
 9. winner_shot_type_F             
10. serve_width_B                  
11. serve_width_BC                 
12. serve_width_BW                 
13. serve_width_C                  
14. serve_width_W                  
15. serve_depth_CTL                
16. serve_depth_NCTL               
17. return_depth_D                 
1

In [40]:
# 保存处理后的数据
print("\n" + "="*60)
print("保存处理后的数据")
print("="*60)

output_file = '2024_Wimbledon_featured_matches_with_indicators.csv'
df_final.to_csv(output_file, index=False)

print(f"\n✓ 数据已保存到: {output_file}")
print(f"✓ 数据形状: {df_final.shape}")
print(f"✓ 包含 {len(df_final.columns)} 列")

# 显示新增指标的示例
print("\n新增指标示例（前3行）:")
metric_cols = [col for col in df_final.columns if any(x in col for x in ['rate', 'count'])]
print(df_final[['match_id', 'player1', 'player2', 'point_no'] + metric_cols[:8]].head(3))


保存处理后的数据

✓ 数据已保存到: 2024_Wimbledon_featured_matches_with_indicators.csv
✓ 数据形状: (7284, 45)
✓ 包含 45 列

新增指标示例（前3行）:
              match_id         player1        player2  point_no  rally_count  \
0  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry         1            2   
1  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry         2            1   
2  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry         3            4   

   p1_1st_serve_in_rate  p2_1st_serve_in_rate  p1_1st_serve_won_rate  \
0                   0.0                   0.0                    0.0   
1                 100.0                   0.0                  100.0   
2                 100.0                   0.0                   50.0   

   p2_1st_serve_won_rate  p1_return_won_rate  p2_return_won_rate  \
0                    0.0                 0.0              100.00   
1                    0.0                 0.0               50.00   
2                    0.0                 0.0               66.67   

 

In [41]:
# 总结处理步骤和新增指标
print("\n" + "="*70)
print("处理总结")
print("="*70)

print("\n✓ 第1步：数据加载")
print(f"  - 原始数据形状: {df.shape}")
print(f"  - 原始列数: {len(df.columns)}")

print("\n✓ 第2步：确定滑动窗口大小 M")
print(f"  - 选择 M = 10 个球")
print(f"  - 原因: 平衡时间跨度和数据充分性")

print("\n✓ 第3步：计算新增指标")
print(f"  - 共计添加 16 个指标")
print(f"  - 各指标针对两位选手(p1, p2)分别计算")

print("\n  新增指标列表:")
new_metrics = [
    ('1st_serve_in_rate', '第一发进球率 (%) - 保留2位小数'),
    ('1st_serve_won_rate', '第一发得分率 (%) - 保留2位小数'),
    ('return_won_rate', '接发球得分率 (%) - 保留2位小数'),
    ('break_count', '破发成功次数 - 最近M行'),
    ('double_fault_count', '双发失误次数 - 最近M行'),
    ('aces_count', 'ACE球次数 - 最近M行'),
    ('winner_count', '制胜球次数 - 最近M行'),
    ('unforced_error_count', '非受迫性失误次数 - 最近M行'),
]
for i, (metric, desc) in enumerate(new_metrics, 1):
    print(f"    {i}. p1_{metric:25s} / p2_{metric}")
    print(f"       └─ {desc}")

print("\n✓ 第4步：删除已转化的原始列")
print(f"  - 删除列数: {len(cols_to_drop)}")
print(f"  - 删除原因: 已转化为新指标，原始列信息已包含")

print("\n  删除的列组:")
print(f"    • 球类统计: ace, winner, double_fault, unforced_error")
print(f"    • 破发相关: break_pt, break_pt_won, break_pt_missed")
print(f"    • 网位进攻: net_pt, net_pt_won")

print("\n✓ 第5步：保存最终数据")
print(f"  - 最终数据形状: {df_final.shape}")
print(f"  - 最终列数: {len(df_final.columns)}")
print(f"  - 文件名: 2024_Wimbledon_featured_matches_with_indicators.csv")

print("\n" + "="*70)
print("数据处理完成！")
print("="*70)


处理总结

✓ 第1步：数据加载
  - 原始数据形状: (7284, 47)
  - 原始列数: 47

✓ 第2步：确定滑动窗口大小 M
  - 选择 M = 10 个球
  - 原因: 平衡时间跨度和数据充分性

✓ 第3步：计算新增指标
  - 共计添加 16 个指标
  - 各指标针对两位选手(p1, p2)分别计算

  新增指标列表:
    1. p1_1st_serve_in_rate         / p2_1st_serve_in_rate
       └─ 第一发进球率 (%) - 保留2位小数
    2. p1_1st_serve_won_rate        / p2_1st_serve_won_rate
       └─ 第一发得分率 (%) - 保留2位小数
    3. p1_return_won_rate           / p2_return_won_rate
       └─ 接发球得分率 (%) - 保留2位小数
    4. p1_break_count               / p2_break_count
       └─ 破发成功次数 - 最近M行
    5. p1_double_fault_count        / p2_double_fault_count
       └─ 双发失误次数 - 最近M行
    6. p1_aces_count                / p2_aces_count
       └─ ACE球次数 - 最近M行
    7. p1_winner_count              / p2_winner_count
       └─ 制胜球次数 - 最近M行
    8. p1_unforced_error_count      / p2_unforced_error_count
       └─ 非受迫性失误次数 - 最近M行

✓ 第4步：删除已转化的原始列
  - 删除列数: 18
  - 删除原因: 已转化为新指标，原始列信息已包含

  删除的列组:
    • 球类统计: ace, winner, double_fault, unforced_error
    • 破发相关: break_pt, break_pt_won

In [42]:
# 最终数据示例和验证
print("\n最终数据样本（某个比赛的前5个球）:")
sample_match = df_final['match_id'].iloc[0]
sample_data = df_final[df_final['match_id'] == sample_match].head(5)

print(f"\n比赛: {sample_match}")
print(f"选手: {sample_data['player1'].iloc[0]} vs {sample_data['player2'].iloc[0]}")
print("\n基础信息:")
print(sample_data[['point_no', 'server', 'point_victor', 'rally_count']].to_string())

print("\n\nPlayer 1 (发球方) 指标:")
p1_metrics = [col for col in sample_data.columns if col.startswith('p1_') and any(x in col for x in ['rate', 'count'])]
print(sample_data[['point_no'] + p1_metrics[:4]].to_string())

print("\n\nPlayer 2 (接发球方) 指标:")
p2_metrics = [col for col in sample_data.columns if col.startswith('p2_') and any(x in col for x in ['rate', 'count'])]
print(sample_data[['point_no'] + p2_metrics[:4]].to_string())

print("\n\n✓ 所有指标已成功计算和添加！")


最终数据样本（某个比赛的前5个球）:

比赛: 2023-wimbledon-1301
选手: Carlos Alcaraz vs Nicolas Jarry

基础信息:
   point_no  server  point_victor  rally_count
0         1       1             2            2
1         2       1             1            1
2         3       1             2            4
3         4       1             1           13
4         5       1             1            1


Player 1 (发球方) 指标:
   point_no  p1_1st_serve_in_rate  p1_1st_serve_won_rate  p1_return_won_rate  p1_break_count
0         1                   0.0                   0.00                 0.0               0
1         2                 100.0                 100.00                 0.0               0
2         3                 100.0                  50.00                 0.0               0
3         4                 100.0                  66.67                 0.0               0
4         5                 100.0                  75.00                 0.0               0


Player 2 (接发球方) 指标:
   point_no  p2_1st_serve_in_

In [43]:
# 最终数据列说明
print("="*70)
print("最终数据列说明")
print("="*70)

print("\n【基础信息列】(7列)")
print("  1. match_id          - 比赛唯一标识符")
print("  2. player1           - 选手1名称")
print("  3. player2           - 选手2名称")
print("  4. set_no            - 当前set编号")
print("  5. game_no           - 当前game编号")
print("  6. point_no          - 当前point编号")
print("  7. server            - 当前发球手 (1=player1, 2=player2)")

print("\n【比赛结果列】(4列)")
print("  8. serve_no          - 第几发 (1 or 2)")
print("  9. point_victor      - 本球赢家 (1=player1, 2=player2)")
print("  10. game_victor      - game赢家")
print("  11. set_victor       - set赢家")

print("\n【发球位置特征列】(one-hot编码, 7列)")
print("  • serve_width_*      - 发球横向位置: B, BC, BW, C, W")
print("  • serve_depth_*      - 发球深度: CTL, NCTL")
print("  • return_depth_*     - 接发深度: D, ND")
print("  • winner_shot_type_* - 制胜球类型: 0, B, F")

print("\n【动态指标列 - 最近10个球统计】(30列)")
print("  --- Player 1 指标 ---")
print("  • p1_1st_serve_in_rate      - 第一发进球率 (%, 保留2位小数)")
print("  • p1_1st_serve_won_rate     - 第一发得分率 (%, 保留2位小数)")
print("  • p1_return_won_rate        - 接发球得分率 (%, 保留2位小数)")
print("  • p1_break_count            - 破发成功次数")
print("  • p1_double_fault_count     - 双发失误次数")
print("  • p1_aces_count             - ACE球次数")
print("  • p1_winner_count           - 制胜球次数")
print("  • p1_unforced_error_count   - 非受迫性失误次数")

print("\n  --- Player 2 指标 ---")
print("  • p2_1st_serve_in_rate      - 第一发进球率 (%, 保留2位小数)")
print("  • p2_1st_serve_won_rate     - 第一发得分率 (%, 保留2位小数)")
print("  • p2_return_won_rate        - 接发球得分率 (%, 保留2位小数)")
print("  • p2_break_count            - 破发成功次数")
print("  • p2_double_fault_count     - 双发失误次数")
print("  • p2_aces_count             - ACE球次数")
print("  • p2_winner_count           - 制胜球次数")
print("  • p2_unforced_error_count   - 非受迫性失误次数")

print("\n【其他统计列】(6列)")
print("  • p1_points_won             - Player1赢得的点数")
print("  • p2_points_won             - Player2赢得的点数")
print("  • rally_count               - 本球的回合数")
print("  • speed_mph                 - 发球速度 (英里/小时)")
print("  • p1_distance_run           - Player1跑动距离")
print("  • p2_distance_run           - Player2跑动距离")

print("\n" + "="*70)
print(f"总计: {len(df_final.columns)} 列")
print("="*70)

最终数据列说明

【基础信息列】(7列)
  1. match_id          - 比赛唯一标识符
  2. player1           - 选手1名称
  3. player2           - 选手2名称
  4. set_no            - 当前set编号
  5. game_no           - 当前game编号
  6. point_no          - 当前point编号
  7. server            - 当前发球手 (1=player1, 2=player2)

【比赛结果列】(4列)
  8. serve_no          - 第几发 (1 or 2)
  9. point_victor      - 本球赢家 (1=player1, 2=player2)
  10. game_victor      - game赢家
  11. set_victor       - set赢家

【发球位置特征列】(one-hot编码, 7列)
  • serve_width_*      - 发球横向位置: B, BC, BW, C, W
  • serve_depth_*      - 发球深度: CTL, NCTL
  • return_depth_*     - 接发深度: D, ND
  • winner_shot_type_* - 制胜球类型: 0, B, F

【动态指标列 - 最近10个球统计】(30列)
  --- Player 1 指标 ---
  • p1_1st_serve_in_rate      - 第一发进球率 (%, 保留2位小数)
  • p1_1st_serve_won_rate     - 第一发得分率 (%, 保留2位小数)
  • p1_return_won_rate        - 接发球得分率 (%, 保留2位小数)
  • p1_break_count            - 破发成功次数
  • p1_double_fault_count     - 双发失误次数
  • p1_aces_count             - ACE球次数
  • p1_winner_count           - 制胜球次数
  • p1_unforc